In [68]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%load_ext autoreload
%autoreload 2

This will be a playground for exploring sklearn's Decision Tree Classifier.

In [2]:
iris = load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [4]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
iris_petal_dims = iris['data'][:, 2:]

In [6]:
type(iris_petal_dims)

numpy.ndarray

In [7]:
iris_dict = {'petal length (cm)': iris_petal_dims[:, 0],
             'petal width (cm)': iris_petal_dims[:, 1],
             'labels': iris['target']}

In [8]:
iris_df = pd.DataFrame(iris_dict)

In [9]:
iris_df

,labels,petal length (cm),petal width (cm)
0,0,1.4,0.2
1,0,1.4,0.2
2,0,1.3,0.2
3,0,1.5,0.2
4,0,1.4,0.2
5,0,1.7,0.4
6,0,1.4,0.3
7,0,1.5,0.2
8,0,1.4,0.2
9,0,1.5,0.1


In [10]:
from collections import Counter
def _gini(labels):
    """
    Calculates the gini impurity for a set of labels.
    """
    total = len(labels)
    label_counts = Counter(labels).values()
    return 1 - sum((p / total)**2
                   for p in label_counts
                   if p)

In [11]:
labels = _gini(iris_df['labels'])

In [12]:
labels

0.6666666666666667

In [13]:
newy = []
minim = []
cost_min = float('inf')
for x in np.arange(.2, 7, .1):
    left = iris_df[iris_df['petal length (cm)'] <= x]['labels']
    right = iris_df[~(iris_df['petal length (cm)'] <= x)]['labels']
    cost = len(left) * _gini(left) / 150 + len(right) * _gini(right) / 150
    if cost < cost_min:
        cost_min = cost
        minim = []
    elif cost == cost_min:
        minim.append(x)
    newy.append(cost)

In [14]:
cost_min

0.3333333333333333

In [15]:
np.mean(minim)

2.4500000000000011

In [16]:
def _gini_cost(labeled_data, feature_name):
    """
    Calculate the cost function as part of the CART algorithm.
    """
    minim = []
    cost_min = float('inf')
    len_labeled_data = len(labeled_data)
    feature_data = [row[0][feature_name] for row in labeled_data]
    max_feature = int(max(feature_data)) * 100
    min_feature = int(min(feature_data)) * 100
    feature_range = max_feature - min_feature
    span = (i / 100 for i in range(min_feature, max_feature + 1))
    for x in span:
        left = []
        right = []
        for i, num in enumerate(feature_data):
            if num <= x:
                left.append(i)
            else:
                right.append(i)
        left_labels = [labeled_data[idx][1] for idx in left]
        right_labels = [labeled_data[idx][1] for idx in right]
        cost = len(left_labels) * _gini(left_labels) / len_labeled_data + len(right_labels) * _gini(right_labels) / len_labeled_data
        if cost < cost_min:
            cost_min = cost
            minim = []
            minim.append(x)
            left_node = [labeled_data[idx] for idx in left]
            right_node = [labeled_data[idx] for idx in right]
        elif cost == cost_min:
            minim.append(x)
    avg_minimums = sum(minim) / len(minim)
    return avg_minimums, left_node, right_node

In [17]:
labels = iris_df['labels'].tolist()

In [18]:
zipped_columns = zip(iris_df['petal length (cm)'].tolist(), iris_df['petal width (cm)'].tolist())

In [19]:
features = [{'petal length (cm)': length,
             'petal width (cm)': width} for length, width in zipped_columns]

In [20]:
zipped_iris_data = list(zip(features, labels))

In [21]:
avgs, left, right =_gini_cost(zipped_iris_data, 'petal length (cm)')

In [22]:
class_labels = [i[1] for i in left]

In [23]:
from collections import Counter

In [24]:
Counter(class_labels)

Counter({0: 50})

In [25]:
listy = [0 if i < 50 else 1 for i in range(100)]

In [26]:
for i in range(50):
    listy.append(2)

In [27]:
_gini(listy)

0.6666666666666667

In [28]:
zero_dtc = DecisionTreeClassifier(max_depth=1)

In [29]:
zero_dtc.fit(iris_petal_dims, iris.target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [30]:
zero_dtc.predict([[5, 1.5]])

array([1])

In [32]:
zipped_iris_data

[({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.3, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.7, 'petal width (cm)': 0.4}, 0),
 ({'petal length (cm)': 1.4, 'petal width (cm)': 0.3}, 0),
 ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.5, 'petal width (cm)': 0.1}, 0),
 ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.6, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.4, 'petal width (cm)': 0.1}, 0),
 ({'petal length (cm)': 1.1, 'petal width (cm)': 0.1}, 0),
 ({'petal length (cm)': 1.2, 'petal width (cm)': 0.2}, 0),
 ({'petal length (cm)': 1.5, 'petal width (cm)': 0.4}, 0),
 ({'petal length (cm)': 1.3, 'petal width (cm)': 0.4}, 0

In [33]:
"""
Module containing the DecisionTree class.
"""
from collections import Counter
from math import log2


class Node:
    """
    Node that contains all information required in order to make predictions and pass
    information on for further evaluation to other nodes.
    """

    def __init__(self, threshold, samples_count, values, classification, feature, gini=None):
        """
        """
        self.threshold = threshold
        self.samples_count = samples_count
        self.values = values
        self.classification = classification
        self.feature = feature
        self.left = None
        self.right = None
        self.gini = gini


class DecisionTree:
    """
    A crude implentation of a decision tree that can either use the Gini index
    or entropy (information gain).
    """

    def __init__(self, max_depth=2):
        """
        """
        self.root = None
        self.max_depth = max_depth

    def train(self, labeled_data, method='gini'):
        """
        """
        if method == 'gini':
            self._cart(labeled_data)
        elif method == 'entropy':
            pass

    def predict(self, data):
        """
        Bsae
        """
        pass

    def _gini(self, labels):
        """
        Calculates the gini impurity for a set of labels.
        """
        total = len(labels)
        label_counts = Counter(labels).values()
        return 1 - sum((p / total)**2
                       for p in label_counts
                       if p)

    def _entropy(self, labels):
        """
        Calculates entropy for a set of labels.
        """
        total = len(labels)
        label_counts = Counter(labels).values()
        return -sum((p / total) * log2(p / total)
                    for p in label_counts
                    if p)

    def _max_gini(self, labels):
        """
        This function used to determine if CART should make a node a leaf node depending on
        how close the gini impurity is to the theoretical max. The max can be represented as follows:
        a_n = 1 - 1/x, x >= 1

        For a given set of labels, the gini impurity will approach the maximum.
        """
        num_labels = len(Counter(labels))
        return 1 - 1/num_labels if num_labels else 0

    def _id3():
        """
        Builds a decision tree using the ID3 algorithm.
        """
        pass

    def _cart(self, labeled_data, depth=0, gini_split_treshold=.25):
        """
        Classification and Regression Tree implementation.
        """
        # Starts with all the data and runs the cost function on each feature to get the best starting split.
        current_depth = depth
        labels = [label[1] for label in labeled_data]
        gini_threshold = gini_split_treshold * self._max_gini(labels)
        if depth >= self.max_depth or self._gini(labels) <= gini_threshold:
            counts = Counter(labels)
            max_val = max(counts.values())
            for key, value in counts.items():
                if counts[key] == max_val:
                    classification = key
            return Node(None, len(labeled_data), labeled_data, classification, None, self._gini(labels))

        lowest_cost = float('inf')
        for feature in labeled_data[0][0].keys():
            gini_calculations = self._gini_cost(labeled_data, feature)
            if gini_calculations[0] < lowest_cost:
                lowest_cost, threshold, left_samples, right_samples = gini_calculations
                chosen_feature = feature

        if self.root is None:
            self.root = Node(threshold, len(labeled_data), labeled_data, left_samples[0][1], chosen_feature, self._gini(labels))
            node = self.root
        else:
            node = Node(threshold, len(labeled_data), labeled_data, left_samples[0][1], chosen_feature, self._gini(labels))

        if current_depth < self.max_depth:
            current_depth += 1
            node.left = self._cart(left_samples, current_depth)
            node.right = self._cart(right_samples, current_depth)
        return node

    def _gini_cost(self, labeled_data, feature_name):
        """
        Calculate the cost function as part of the CART algorithm.

        For continuous data, multiplying by 100 then dividing the individual numbers by 100 was an operation
        chosen arbitrarily to find the best number to split the data on. It's a magic number for sure.
        """
        minim = []
        cost_min = float('inf')
        len_labeled_data = len(labeled_data)
        feature_data = [row[0][feature_name] for row in labeled_data]
        max_feature = int(max(feature_data)) * 100
        min_feature = int(min(feature_data)) * 100
        span = (i / 100 for i in range(min_feature, max_feature + 1))
        for x in span:
            left = []
            right = []
            for i, num in enumerate(feature_data):
                if num <= x:
                    left.append(i)
                else:
                    right.append(i)
            left_labels = [labeled_data[idx][1] for idx in left]
            right_labels = [labeled_data[idx][1] for idx in right]
            cost = len(left_labels) * self._gini(left_labels) / len_labeled_data + len(right_labels) * self._gini(right_labels) / len_labeled_data
            if cost < cost_min:
                cost_min = cost
                minim = []
                minim.append(x)
                left_samples = [labeled_data[idx] for idx in left]
                right_samples = [labeled_data[idx] for idx in right]
            elif cost == cost_min:
                minim.append(x)
        avg_minimums = sum(minim) / len(minim)
        return cost_min, avg_minimums, left_samples, right_samples


'\nModule containing the DecisionTree class.\n'

In [34]:
dct = DecisionTree()

In [35]:
dct._cart(zipped_iris_data)

In [36]:
dct.root.__dict__

{'classification': 0,
 'feature': 'petal length (cm)',
 'gini': 0.6666666666666667,
 'left': <__main__.Node at 0x1cd98ed55f8>,
 'right': <__main__.Node at 0x1cd98ed59b0>,
 'samples_count': 150,
 'threshold': 2.4450000000000003,
 'values': [({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.3, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.7, 'petal width (cm)': 0.4}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.3}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.1}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.6, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width

In [37]:
dct.root.right.right.right

In [38]:
dct.root.left.right

In [39]:
dct.root.right.left.left

In [40]:
dct.root.right

In [41]:
new_listy = []
for x in range(4):
    for i in range(50):
        new_listy.append(x)

In [42]:
dct.root.__dict__

{'classification': 0,
 'feature': 'petal length (cm)',
 'gini': 0.6666666666666667,
 'left': <__main__.Node at 0x1cd98ed55f8>,
 'right': <__main__.Node at 0x1cd98ed59b0>,
 'samples_count': 150,
 'threshold': 2.4450000000000003,
 'values': [({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.3, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.7, 'petal width (cm)': 0.4}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.3}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.1}, 0),
  ({'petal length (cm)': 1.5, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.6, 'petal width (cm)': 0.2}, 0),
  ({'petal length (cm)': 1.4, 'petal width

In [43]:
5/6

0.8333333333333334

In [44]:
.1 * 5/6

0.08333333333333333

In [45]:
5/6 - (.05 * 5/6)

0.7916666666666667

In [46]:
len(Counter(new_listy))

4

In [47]:
dct._max_gini([1])

0.0

In [48]:
new_listy.append(2)

In [49]:
max(Counter(new_listy).values())

51

In [50]:
new_listy

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2]

In [52]:
import sys
sys.path

['',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\python36.zip',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\DLLs',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages\\Babel-2.5.0-py3.6.egg',
 'c:\\users\\kurtrm\\projects\\decision_tree',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages\\win32',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\kurtrm\\Anaconda3\\envs\\ENV\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\kurtrm\\.ipython']

In [54]:
from src.decision_tree import DecisionTree

In [94]:
dt = DecisionTree()

In [56]:
dt.predict(iris_petal_dims)

In [57]:
dt.root

In [69]:
from src.decision_tree import DecisionTree

In [122]:
dt3 = DecisionTree()

In [123]:
dt3.train(zipped_iris_data)

In [108]:
attempts = [
    {'petal length (cm)': 1.4, 'petal width (cm)': 0.2},
    {'petal length (cm)': 1.4, 'petal width (cm)': 0.2},
    {'petal length (cm)': 1.3, 'petal width (cm)': 0.2},
    {'petal length (cm)': 1.5, 'petal width (cm)': 0.2},
    {'petal length (cm)': 1.4, 'petal width (cm)': 0.2},
    {'petal length (cm)': 5.5, 'petal width (cm)': 1.8},
    {'petal length (cm)': 4.5, 'petal width (cm)': 1.6}
    
]

In [105]:
dt3.predict(attempts)

[0, 0, 0, 0, 0, 2, 1]

In [101]:
dt3.root.right

In [67]:
dt3.root.left.left

In [91]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'],
      dtype='<U10')

In [124]:
print(dt3.root)


    petal length (cm) <= 2.445
    gini = 0.667
    samples = 150
    values = [50, 50, 50]
    class = 0


In [125]:
dt3.root

<[Node] gini=0.6666666666666667 feature=petal length (cm)>